<a href="https://colab.research.google.com/github/Judy-Choi/Tensorflow_Certificate/blob/main/15_diesel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# val_mae = 0.026

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.10.0

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.9/dist-packages/tensorflow-2.12.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.12.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━

# 정답

In [ ]:
#release
# ==============================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative to its
# difficulty. So your Category 1 question will score significantly less than
# your Category 5 question.
#
# WARNING: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure. You do not need them to solve the question.
#
# WARNING: If you are using the GRU layer, it is advised not to use the
# recurrent_dropout argument (you can alternatively set it to 0),
# since it has not been implemented in the cuDNN kernel and may
# result in much longer training times.
#
# WARNING: Input and output shape requirements are laid down in the section 
# 'INSTRUCTIONS' below and also reiterated in code comments. 
# Please read them thoroughly. After submitting the trained model for scoring, 
# if you are receiving a score of 0 or an error, please recheck the input and 
# output shapes of the model to see if it exactly matches our requirements. 
# Grading infrastrcuture is very strict about the shape requirements. Most common 
# issues occur when the shapes are not matching our expectations.
#
# TIP: You can print the output of model.summary() to review the model
# architecture, input and output shapes of each layer.
# If you have made sure that you have matched the shape requirements
# and all the other instructions we have laid down, and still
# receive a bad score, you must work on improving your model.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ==============================================================================
#
# TIME SERIES QUESTION
#
# Build and train a neural network to predict the time indexed variable of
# the univariate US diesel prices (On - Highway) All types for the period of
# 1994 - 2021.
# Using a window of past 10 observations of 1 feature , train the model
# to predict the next 10 observations of that feature.
#
# ==============================================================================
#
# ABOUT THE DATASET
#
# Original Source:
# https://www.eia.gov/dnav/pet/pet_pri_gnd_dcus_nus_w.htm#
#
# For the purpose of the examination we have used the Diesel (On - Highway) -
# All Types time series data for the period of 1994 - 2021 from the
# aforementioned link. The dataset has 1 time indexed feature.
# We have provided a cleaned version of the data.
#
# ==============================================================================
#
# INSTRUCTIONS
#
# Complete the code in following functions:
# 1. solution_model()
#
# You may receive a score of 0 or your code will fail to be graded if the 
# following criteria are not met:
#
# 1. Model input shape must be (BATCH_SIZE, N_PAST = 10, N_FEATURES = 1),
#    since the testing infrastructure expects a window of past N_PAST = 10
#    observations of the 1 feature to predict the next N_FUTURE = 10
#    observations of the same feature.
#
# 2. Model output shape must be (BATCH_SIZE, N_FUTURE = 10, N_FEATURES = 1)
#
# 3. The last layer of your model must be a Dense layer with 1 neuron since
#    the model is expected to predict observations of 1 feature.
#
# 4. Don't change the values of the following constants:
#    SPLIT_TIME, N_FEATURES, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT, in
#    solution_model() (See code for additional note on BATCH_SIZE).
#
# 5. Code for normalizing the data is provided - don't change it.
#    Changing the normalizing code will affect your score.
#
# 6. Code for converting the dataset into windows is provided - don't change it.
#    Changing the windowing code will affect your score.
#
# 7. Code for setting the seed is provided - don't change it.
#
# Make sure that the model architecture and input, output shapes match our
# requirements by printing model.summary() and reviewing its output.
#
# HINT: If you follow all the rules mentioned above and throughout this
# question while training your neural network, there is a possibility that a
# validation MAE of approximately 0.02 or less on the normalized validation
# dataset may fetch you top marks.


import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data


# This function is used to map the time series dataset into windows of
# features and respective targets, to prepare it for training and validation.
# The first element of the first window will be the first element of
# the dataset.
#
# Consecutive windows are constructed by shifting the starting position
# of the first window forward, one at a time (indicated by shift=1).
#
# For a window of n_past number of observations of the time
# indexed variable in the dataset, the target for the window is the next
# n_future number of observations of the variable, after the
# end of the window.

# DO NOT CHANGE THIS.
def windowed_dataset(series, batch_size, n_past=10, n_future=10, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)


# This function loads the data from the CSV file, normalizes the data and
# splits the dataset into train and validation data. It also uses
# windowed_dataset() to split the data into windows of observations and
# targets. Finally it defines, compiles and trains a neural network. This
# function returns the final trained model.

# COMPLETE THE CODE IN THIS FUNCTION
def solution_model():
    # DO NOT CHANGE THIS CODE
    # Reads the dataset.
    df = pd.read_csv('/content/drive/MyDrive/WISET/deep_learning/5. diesel/Weekly_U.S.Diesel_Retail_Prices.csv',
                     infer_datetime_format=True, index_col='Week of', header=0)

    # Number of features in the dataset. We use all features as predictors to
    # predict all features of future time steps.
    N_FEATURES = len(df.columns) # DO NOT CHANGE THIS

    # Normalizes the data
    data = df.values
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))

    # Splits the data into training and validation sets.
    SPLIT_TIME = int(len(data) * 0.8) # DO NOT CHANGE THIS
    x_train = data[:SPLIT_TIME]
    x_valid = data[SPLIT_TIME:]

    # DO NOT CHANGE THIS CODE
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    # DO NOT CHANGE BATCH_SIZE IF YOU ARE USING STATEFUL LSTM/RNN/GRU.
    # THE TEST WILL FAIL TO GRADE YOUR SCORE IN SUCH CASES.
    # In other cases, it is advised not to change the batch size since it
    # might affect your final scores. While setting it to a lower size
    # might not do any harm, higher sizes might affect your scores.
    BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS

    # DO NOT CHANGE N_PAST, N_FUTURE, SHIFT. The tests will fail to run
    # on the server.
    # Number of past time steps based on which future observations should be
    # predicted
    N_PAST = 10  # DO NOT CHANGE THIS

    # Number of future time steps which are to be predicted.
    N_FUTURE = 10  # DO NOT CHANGE THIS

    # By how many positions the window slides to create a new window
    # of observations.
    SHIFT = 1  # DO NOT CHANGE THIS

    # Code to create windowed train and validation datasets.
    train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

    # Code to define your model.
    model = tf.keras.models.Sequential([

        # ADD YOUR LAYERS HERE.
        Conv1D(filters=32, kernel_size=5, padding='causal', activation='relu', input_shape=[N_PAST, 1]),
        Bidirectional(LSTM(32, return_sequences=True)),
        Bidirectional(LSTM(32, return_sequences=True)),
        Dense(256, activation='relu'),
        Dense(16, activation='relu'),
        # If you don't follow the instructions in the following comments,
        # tests will fail to grade your code:
        # The input layer of your model must have an input shape of:
        # (BATCH_SIZE, N_PAST = 10, N_FEATURES = 1)
        # The model must have an output shape of:
        # (BATCH_SIZE, N_FUTURE = 10, N_FEATURES = 1).
        # Make sure that there are N_FEATURES = 1 neurons in the final dense
        # layer since the model predicts 1 feature.

        # HINT: Bidirectional LSTMs may help boost your score. This is only a
        # suggestion.

        # WARNING: After submitting the trained model for scoring, if you are
        # receiving a score of 0 or an error, please recheck the input and 
        # output shapes of the model to see if it exactly matches our requirements. 
        # The grading infrastructure is very strict about the shape requirements. 
        # Most common issues occur when the shapes are not matching our 
        # expectations.
        #
        # TIP: You can print the output of model.summary() to review the model
        # architecture, input and output shapes of each layer.
        # If you have made sure that you have matched the shape requirements
        # and all the other instructions we have laid down, and still
        # receive a bad score, you must work on improving your model.

        # WARNING: If you are using the GRU layer, it is advised not to use the
        # recurrent_dropout argument (you can alternatively set it to 0),
        # since it has not been implemented in the cuDNN kernel and may
        # result in much longer training times.
        tf.keras.layers.Dense(N_FEATURES)
    ])

    # Code to train and compile the model
    checkpoint_path='model/my_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_mae',
                             verbose=1,
                             )
    early_stopping = EarlyStopping(monitor='val_mae', patience=10)
   
    optimizer = tf.keras.optimizers.Adam(0.0001)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])

    model.fit(train_set,
              validation_data=(valid_set),
              epochs=100,
              callbacks=[checkpoint, early_stopping])

    model.load_weights(checkpoint_path)

    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.

if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")


# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS

#def model_forecast(model, series, window_size, batch_size):
#    ds = tf.data.Dataset.from_tensor_slices(series)
#    ds = ds.window(window_size, shift=1, drop_remainder=True)
#    ds = ds.flat_map(lambda w: w.batch(window_size))
#    ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)
#    forecast = model.predict(ds)
#    return forecast

# PASS THE NORMALIZED data IN THE FOLLOWING CODE

# rnn_forecast = model_forecast(model, data, N_PAST, BATCH_SIZE)
# rnn_forecast = rnn_forecast[SPLIT_TIME - N_PAST:-1, 0, 0]

# x_valid = np.squeeze(x_valid[:rnn_forecast.shape[0]])
# result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()


Epoch 1/100
     34/Unknown - 11s 53ms/step - loss: 0.0504 - mae: 0.2440
Epoch 1: val_mae improved from inf to 0.29766, saving model to model/my_checkpoint.ckpt
35/35 [==============================] - 14s 146ms/step - loss: 0.0500 - mae: 0.2446 - val_loss: 0.0463 - val_mae: 0.2977
Epoch 2/100
33/35 [===========================>..] - ETA: 0s - loss: 0.0280 - mae: 0.1705
Epoch 2: val_mae improved from 0.29766 to 0.17731, saving model to model/my_checkpoint.ckpt
35/35 [==============================] - 1s 34ms/step - loss: 0.0277 - mae: 0.1719 - val_loss: 0.0174 - val_mae: 0.1773
Epoch 3/100
33/35 [===========================>..] - ETA: 0s - loss: 0.0124 - mae: 0.1331
Epoch 3: val_mae improved from 0.17731 to 0.05640, saving model to model/my_checkpoint.ckpt
35/35 [==============================] - 1s 35ms/step - loss: 0.0119 - mae: 0.1291 - val_loss: 0.0025 - val_mae: 0.0564
Epoch 4/100
35/35 [==============================] - ETA: 0s - loss: 0.0097 - mae: 0.1237
Epoch 4: val_mae improv

# ChatGPT

Q) 다음 코드의 성능을 개선해 줘.
사용하는 데이터는 float 형의 시계열 데이터이고
다음 조건을 적용해 줘.
Use dropout
Use batch normalization
Use checkpoint with save best only, and save weight only and verbose = 1
Use early stopping with patience = 10
Use reduce learning rate on plateau
set verbose = 1
Set epochs = 100
do not use data augmentation
Use other methods to make val loss value lower.

A) 여러 방법을 사용하여 모델 성능을 개선할 수 있습니다. 제안된 조건을 따르면서 다음과 같은 방법을 시도할 수 있습니다.

Batch Normalization: 모델의 안정성을 높일 수 있는 Batch Normalization 레이어를 추가합니다.
Dropout: 과적합을 방지하기 위해 Dropout 레이어를 추가합니다.
Learning Rate Schedule: ReduceLROnPlateau 콜백을 사용하여 학습률을 조정합니다.
Optimizer: 모델의 최적화 방법으로 Adam optimizer를 사용합니다.

In [ ]:
#release
# ==============================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative to its
# difficulty. So your Category 1 question will score significantly less than
# your Category 5 question.
#
# WARNING: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure. You do not need them to solve the question.
#
# WARNING: If you are using the GRU layer, it is advised not to use the
# recurrent_dropout argument (you can alternatively set it to 0),
# since it has not been implemented in the cuDNN kernel and may
# result in much longer training times.
#
# WARNING: Input and output shape requirements are laid down in the section 
# 'INSTRUCTIONS' below and also reiterated in code comments. 
# Please read them thoroughly. After submitting the trained model for scoring, 
# if you are receiving a score of 0 or an error, please recheck the input and 
# output shapes of the model to see if it exactly matches our requirements. 
# Grading infrastrcuture is very strict about the shape requirements. Most common 
# issues occur when the shapes are not matching our expectations.
#
# TIP: You can print the output of model.summary() to review the model
# architecture, input and output shapes of each layer.
# If you have made sure that you have matched the shape requirements
# and all the other instructions we have laid down, and still
# receive a bad score, you must work on improving your model.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ==============================================================================
#
# TIME SERIES QUESTION
#
# Build and train a neural network to predict the time indexed variable of
# the univariate US diesel prices (On - Highway) All types for the period of
# 1994 - 2021.
# Using a window of past 10 observations of 1 feature , train the model
# to predict the next 10 observations of that feature.
#
# ==============================================================================
#
# ABOUT THE DATASET
#
# Original Source:
# https://www.eia.gov/dnav/pet/pet_pri_gnd_dcus_nus_w.htm#
#
# For the purpose of the examination we have used the Diesel (On - Highway) -
# All Types time series data for the period of 1994 - 2021 from the
# aforementioned link. The dataset has 1 time indexed feature.
# We have provided a cleaned version of the data.
#
# ==============================================================================
#
# INSTRUCTIONS
#
# Complete the code in following functions:
# 1. solution_model()
#
# You may receive a score of 0 or your code will fail to be graded if the 
# following criteria are not met:
#
# 1. Model input shape must be (BATCH_SIZE, N_PAST = 10, N_FEATURES = 1),
#    since the testing infrastructure expects a window of past N_PAST = 10
#    observations of the 1 feature to predict the next N_FUTURE = 10
#    observations of the same feature.
#
# 2. Model output shape must be (BATCH_SIZE, N_FUTURE = 10, N_FEATURES = 1)
#
# 3. The last layer of your model must be a Dense layer with 1 neuron since
#    the model is expected to predict observations of 1 feature.
#
# 4. Don't change the values of the following constants:
#    SPLIT_TIME, N_FEATURES, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT, in
#    solution_model() (See code for additional note on BATCH_SIZE).
#
# 5. Code for normalizing the data is provided - don't change it.
#    Changing the normalizing code will affect your score.
#
# 6. Code for converting the dataset into windows is provided - don't change it.
#    Changing the windowing code will affect your score.
#
# 7. Code for setting the seed is provided - don't change it.
#
# Make sure that the model architecture and input, output shapes match our
# requirements by printing model.summary() and reviewing its output.
#
# HINT: If you follow all the rules mentioned above and throughout this
# question while training your neural network, there is a possibility that a
# validation MAE of approximately 0.02 or less on the normalized validation
# dataset may fetch you top marks.


import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data


# This function is used to map the time series dataset into windows of
# features and respective targets, to prepare it for training and validation.
# The first element of the first window will be the first element of
# the dataset.
#
# Consecutive windows are constructed by shifting the starting position
# of the first window forward, one at a time (indicated by shift=1).
#
# For a window of n_past number of observations of the time
# indexed variable in the dataset, the target for the window is the next
# n_future number of observations of the variable, after the
# end of the window.

# DO NOT CHANGE THIS.
def windowed_dataset(series, batch_size, n_past=10, n_future=10, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)


# This function loads the data from the CSV file, normalizes the data and
# splits the dataset into train and validation data. It also uses
# windowed_dataset() to split the data into windows of observations and
# targets. Finally it defines, compiles and trains a neural network. This
# function returns the final trained model.

# COMPLETE THE CODE IN THIS FUNCTION
def solution_model():
    # DO NOT CHANGE THIS CODE
    # Reads the dataset.
    df = pd.read_csv('/content/drive/MyDrive/TF_WISET/Weekly_U.S.Diesel_Retail_Prices.csv',
                     infer_datetime_format=True, index_col='Week of', header=0)

    # Number of features in the dataset. We use all features as predictors to
    # predict all features of future time steps.
    N_FEATURES = len(df.columns) # DO NOT CHANGE THIS

    # Normalizes the data
    data = df.values
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))

    # Splits the data into training and validation sets.
    SPLIT_TIME = int(len(data) * 0.8) # DO NOT CHANGE THIS
    x_train = data[:SPLIT_TIME]
    x_valid = data[SPLIT_TIME:]

    # DO NOT CHANGE THIS CODE
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    # DO NOT CHANGE BATCH_SIZE IF YOU ARE USING STATEFUL LSTM/RNN/GRU.
    # THE TEST WILL FAIL TO GRADE YOUR SCORE IN SUCH CASES.
    # In other cases, it is advised not to change the batch size since it
    # might affect your final scores. While setting it to a lower size
    # might not do any harm, higher sizes might affect your scores.
    BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS

    # DO NOT CHANGE N_PAST, N_FUTURE, SHIFT. The tests will fail to run
    # on the server.
    # Number of past time steps based on which future observations should be
    # predicted
    N_PAST = 10  # DO NOT CHANGE THIS

    # Number of future time steps which are to be predicted.
    N_FUTURE = 10  # DO NOT CHANGE THIS

    # By how many positions the window slides to create a new window
    # of observations.
    SHIFT = 1  # DO NOT CHANGE THIS

    # Code to create windowed train and validation datasets.
    train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

    # Code to define your model.
    model = tf.keras.models.Sequential([
        Conv1D(filters=32, kernel_size=5, padding='causal', activation='relu', input_shape=[N_PAST, 1]),
        Bidirectional(LSTM(32, return_sequences=True)),
        BatchNormalization(),
        Dropout(0.2),
        Bidirectional(LSTM(32, return_sequences=True)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dense(16, activation='relu'),
        Dense(N_FEATURES)
    ])

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

    # Define callbacks
    checkpoint_path = 'model/my_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path, save_weights_only=True, save_best_only=True, monitor='val_mae', verbose=1)
    early_stopping = EarlyStopping(monitor='val_mae', patience=10)
    reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=5, verbose=1)

    # Train the model
    history = model.fit(train_set, epochs=100, validation_data=valid_set, callbacks=[checkpoint, early_stopping, reduce_lr], verbose=1)

    model.load_weights(checkpoint_path)

    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.

if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

Epoch 1/100
     35/Unknown - 13s 31ms/step - loss: 0.7551 - mae: 0.6841
Epoch 1: val_mae improved from inf to 0.48094, saving model to model/my_checkpoint.ckpt
35/35 [==============================] - 15s 93ms/step - loss: 0.7551 - mae: 0.6841 - val_loss: 0.2361 - val_mae: 0.4809 - lr: 1.0000e-04
Epoch 2/100
35/35 [==============================] - ETA: 0s - loss: 0.3979 - mae: 0.4845
Epoch 2: val_mae did not improve from 0.48094
35/35 [==============================] - 1s 37ms/step - loss: 0.3979 - mae: 0.4845 - val_loss: 0.2945 - val_mae: 0.5380 - lr: 1.0000e-04
Epoch 3/100
35/35 [==============================] - ETA: 0s - loss: 0.2880 - mae: 0.4216
Epoch 3: val_mae did not improve from 0.48094
35/35 [==============================] - 1s 37ms/step - loss: 0.2880 - mae: 0.4216 - val_loss: 0.3413 - val_mae: 0.5798 - lr: 1.0000e-04
Epoch 4/100
35/35 [==============================] - ETA: 0s - loss: 0.2294 - mae: 0.3754
Epoch 4: val_mae did not improve from 0.48094
35/35 [============

# 직접 코딩

과제 순서를 바꿔서, ChatGPT 가 짜 준 코드를 직접 개선해 본다

In [ ]:
#release
# ==============================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative to its
# difficulty. So your Category 1 question will score significantly less than
# your Category 5 question.
#
# WARNING: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure. You do not need them to solve the question.
#
# WARNING: If you are using the GRU layer, it is advised not to use the
# recurrent_dropout argument (you can alternatively set it to 0),
# since it has not been implemented in the cuDNN kernel and may
# result in much longer training times.
#
# WARNING: Input and output shape requirements are laid down in the section 
# 'INSTRUCTIONS' below and also reiterated in code comments. 
# Please read them thoroughly. After submitting the trained model for scoring, 
# if you are receiving a score of 0 or an error, please recheck the input and 
# output shapes of the model to see if it exactly matches our requirements. 
# Grading infrastrcuture is very strict about the shape requirements. Most common 
# issues occur when the shapes are not matching our expectations.
#
# TIP: You can print the output of model.summary() to review the model
# architecture, input and output shapes of each layer.
# If you have made sure that you have matched the shape requirements
# and all the other instructions we have laid down, and still
# receive a bad score, you must work on improving your model.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ==============================================================================
#
# TIME SERIES QUESTION
#
# Build and train a neural network to predict the time indexed variable of
# the univariate US diesel prices (On - Highway) All types for the period of
# 1994 - 2021.
# Using a window of past 10 observations of 1 feature , train the model
# to predict the next 10 observations of that feature.
#
# ==============================================================================
#
# ABOUT THE DATASET
#
# Original Source:
# https://www.eia.gov/dnav/pet/pet_pri_gnd_dcus_nus_w.htm#
#
# For the purpose of the examination we have used the Diesel (On - Highway) -
# All Types time series data for the period of 1994 - 2021 from the
# aforementioned link. The dataset has 1 time indexed feature.
# We have provided a cleaned version of the data.
#
# ==============================================================================
#
# INSTRUCTIONS
#
# Complete the code in following functions:
# 1. solution_model()
#
# You may receive a score of 0 or your code will fail to be graded if the 
# following criteria are not met:
#
# 1. Model input shape must be (BATCH_SIZE, N_PAST = 10, N_FEATURES = 1),
#    since the testing infrastructure expects a window of past N_PAST = 10
#    observations of the 1 feature to predict the next N_FUTURE = 10
#    observations of the same feature.
#
# 2. Model output shape must be (BATCH_SIZE, N_FUTURE = 10, N_FEATURES = 1)
#
# 3. The last layer of your model must be a Dense layer with 1 neuron since
#    the model is expected to predict observations of 1 feature.
#
# 4. Don't change the values of the following constants:
#    SPLIT_TIME, N_FEATURES, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT, in
#    solution_model() (See code for additional note on BATCH_SIZE).
#
# 5. Code for normalizing the data is provided - don't change it.
#    Changing the normalizing code will affect your score.
#
# 6. Code for converting the dataset into windows is provided - don't change it.
#    Changing the windowing code will affect your score.
#
# 7. Code for setting the seed is provided - don't change it.
#
# Make sure that the model architecture and input, output shapes match our
# requirements by printing model.summary() and reviewing its output.
#
# HINT: If you follow all the rules mentioned above and throughout this
# question while training your neural network, there is a possibility that a
# validation MAE of approximately 0.02 or less on the normalized validation
# dataset may fetch you top marks.


import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data


# This function is used to map the time series dataset into windows of
# features and respective targets, to prepare it for training and validation.
# The first element of the first window will be the first element of
# the dataset.
#
# Consecutive windows are constructed by shifting the starting position
# of the first window forward, one at a time (indicated by shift=1).
#
# For a window of n_past number of observations of the time
# indexed variable in the dataset, the target for the window is the next
# n_future number of observations of the variable, after the
# end of the window.

# DO NOT CHANGE THIS.
def windowed_dataset(series, batch_size, n_past=10, n_future=10, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)


# This function loads the data from the CSV file, normalizes the data and
# splits the dataset into train and validation data. It also uses
# windowed_dataset() to split the data into windows of observations and
# targets. Finally it defines, compiles and trains a neural network. This
# function returns the final trained model.

# COMPLETE THE CODE IN THIS FUNCTION
def solution_model():
    # DO NOT CHANGE THIS CODE
    # Reads the dataset.
    df = pd.read_csv('/content/drive/MyDrive/TF_WISET/Weekly_U.S.Diesel_Retail_Prices.csv',
                     infer_datetime_format=True, index_col='Week of', header=0)

    # Number of features in the dataset. We use all features as predictors to
    # predict all features of future time steps.
    N_FEATURES = len(df.columns) # DO NOT CHANGE THIS

    # Normalizes the data
    data = df.values
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))

    # Splits the data into training and validation sets.
    SPLIT_TIME = int(len(data) * 0.8) # DO NOT CHANGE THIS
    x_train = data[:SPLIT_TIME]
    x_valid = data[SPLIT_TIME:]

    # DO NOT CHANGE THIS CODE
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    # DO NOT CHANGE BATCH_SIZE IF YOU ARE USING STATEFUL LSTM/RNN/GRU.
    # THE TEST WILL FAIL TO GRADE YOUR SCORE IN SUCH CASES.
    # In other cases, it is advised not to change the batch size since it
    # might affect your final scores. While setting it to a lower size
    # might not do any harm, higher sizes might affect your scores.
    BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS

    # DO NOT CHANGE N_PAST, N_FUTURE, SHIFT. The tests will fail to run
    # on the server.
    # Number of past time steps based on which future observations should be
    # predicted
    N_PAST = 10  # DO NOT CHANGE THIS

    # Number of future time steps which are to be predicted.
    N_FUTURE = 10  # DO NOT CHANGE THIS

    # By how many positions the window slides to create a new window
    # of observations.
    SHIFT = 1  # DO NOT CHANGE THIS

    # Code to create windowed train and validation datasets.
    train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

    # Code to define your model.
    model = tf.keras.models.Sequential([
        Conv1D(filters=32, kernel_size=5, padding='causal', activation='relu', input_shape=[N_PAST, 1]),
        Bidirectional(LSTM(32, return_sequences=True)),
        Bidirectional(LSTM(32, return_sequences=True)),
        Dense(256, activation='relu'),
        Dense(16, activation='relu'),
        Dense(N_FEATURES)
    ])

    # Compile the model
    # optimizer = SGD 로 교체 시 val_mae 가 일정하지만 너무 느리게 감소함.
    # optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

    # Define callbacks
    checkpoint_path = 'model/my_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path, save_weights_only=True, save_best_only=True, monitor='val_mae', verbose=1)
    early_stopping = EarlyStopping(monitor='val_mae', patience=10)
    reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=5, verbose=1)

    # Train the model
    history = model.fit(train_set, epochs=150, validation_data=valid_set, callbacks=[checkpoint, early_stopping, reduce_lr], verbose=1)

    model.load_weights(checkpoint_path)

    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.

if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

Epoch 1/150
     35/Unknown - 10s 46ms/step - loss: 0.1237 - mae: 0.2732
Epoch 1: val_mae improved from inf to 0.37470, saving model to model/my_checkpoint.ckpt
35/35 [==============================] - 13s 118ms/step - loss: 0.1237 - mae: 0.2732 - val_loss: 0.1448 - val_mae: 0.3747 - lr: 1.0000e-04
Epoch 2/150
34/35 [============================>.] - ETA: 0s - loss: 0.1202 - mae: 0.2647
Epoch 2: val_mae improved from 0.37470 to 0.36767, saving model to model/my_checkpoint.ckpt
35/35 [==============================] - 1s 32ms/step - loss: 0.1199 - mae: 0.2666 - val_loss: 0.1396 - val_mae: 0.3677 - lr: 1.0000e-04
Epoch 3/150
34/35 [============================>.] - ETA: 0s - loss: 0.1166 - mae: 0.2583
Epoch 3: val_mae improved from 0.36767 to 0.36072, saving model to model/my_checkpoint.ckpt
35/35 [==============================] - 1s 34ms/step - loss: 0.1163 - mae: 0.2602 - val_loss: 0.1345 - val_mae: 0.3607 - lr: 1.0000e-04
Epoch 4/150
35/35 [==============================] - ETA: 0s -